In [235]:
import pyaudio
import numpy as np

# https://mathoverflow.net/questions/369751/boolean-ring-of-unitary-divisors-structure-of-unitary-divisors
from sage.all import * 

def unitary_divisors(n):
    return [x for x in divisors(n) if gcd(x,n/x)==1]

def kk(a,b):
    return gcd(a,b)**2/(a*b)

def plus(a,b):
    return a*b/(gcd(a,b)**2)

def mult(a,b):
    return gcd(a,b)

def compositionToDivisor(c):
    s = Composition(c).to_subset()
    if len(s)==0:
        return 1
    return prod([nth_prime(x) for x in s])

def rad(n):
    return prod(prime_divisors(n))

def divisorToComposition(d,n):
    if rad(d)==d:
        s = set([prime_pi(x) for x in prime_divisors(d)])
        return Composition(from_subset=(s,n))

def compositionKernel(c1,c2):
    a = compositionToDivisor(c1)
    b = compositionToDivisor(c2)
    return kk(a,b)

def primorial(n):
    return prod(primes(nth_prime(n)))


def pivoted_cholesky(M,max_rank=int(30)):
    import tensorflow_probability as tfp
    import tensorflow as tf
    import numpy as np
    piv_chol = (tfp.math.pivoted_cholesky(M,max_rank = max_rank, diag_rtol=float(0.001), name=None)).numpy()
    return piv_chol


# https://mathoverflow.net/questions/373475/a-geometric-approach-to-the-odd-perfect-number-problem
# embedding:
def jordanTotient(n,k=2):
    return n**2*prod([1-1/p**2 for p in prime_divisors(n)])

# embedding:
# https://mathoverflow.net/questions/373475/a-geometric-approach-to-the-odd-perfect-number-problem
def phi(C,axis=0):
    from scipy.sparse import dok_matrix,lil_matrix,csr_matrix,csc_matrix
    import numpy as np
    n = int(np.sum(list(C)))
    print("n = ",n)
    A = compositionToDivisor(C)
    print("A =",A)
    print("P_n = ",primorial(n-1))
    if axis == 1:
        S = csr_matrix((int(1),int(2**(n-1))), dtype=np.float64)
        for d in divisors(A):
            ds = divisorToComposition(d,n).to_subset()
            x = int(np.sum([2**(i-1) for i in ds]))            
            S[0,x]         = float(1.0/A)*np.sqrt(jordanTotient(d,k=2))
    elif axis == 0:
        S = dok_matrix((int(2**(n-1)),int(1)), dtype=np.float64)
        for d in divisors(A):
            ds = divisorToComposition(d,n).to_subset()
            x = int(np.sum([2**(i-1) for i in ds]))            
            S[x,0]         = float(1.0/A)*np.sqrt(jordanTotient(d,k=2))
    return S    

# embedding of all Compositions(n) in a dok_matrix
def phiN(n,maxN=8):
    from scipy.sparse import dok_matrix,lil_matrix,csr_matrix,csc_matrix
    import numpy as np
    if n > maxN:
        return None
    CC = Compositions(n)
    lC = 2**(n-1)
    S = dok_matrix((lC,lC),dtype=np.float64)
    #print(S.shape)
    c = 0
    #print(CC)
    for C in CC:
        #print(C)
        A = compositionToDivisor(list(C))
        for d in divisors(A):
            ds = divisorToComposition(d,n).to_subset()
            x = int(np.sum([2**(i-1) for i in ds]))
            S[c,x] = float(1.0/A)*np.sqrt(jordanTotient(d,k=2))
        c+=1    
    return S

def embN(n,maxN=8):
    from scipy.sparse import dok_matrix,lil_matrix,csr_matrix,csc_matrix
    import numpy as np    
    if n > maxN:
        return None
    CC = Compositions(n)
    lC = 2**(n-1)
    S = dok_matrix((n,lC),dtype=np.float64)
    c =0
    for C in CC:
        sC = C.to_subset()
        for s in sC:
            S[s-1,c] = 1
        c+=1    
    return S.transpose()    

def emb(C,axis=1):
    from scipy.sparse import dok_matrix,lil_matrix,csr_matrix,csc_matrix
    import numpy as np    
    n = sum(C)
    lC = 2**(n-1)
    print(n,lC)
    if axis == 0:
        S = dok_matrix((lC,1),dtype=np.float64)
        sC = Composition(C).to_subset()
        for s in sC:
            S[s-1,0] = 1
    if axis == 1:
        S = dok_matrix((1,lC),dtype=np.float64)
        sC = Composition(C).to_subset()
        for s in sC:
            S[0,s-1] = 1
    return S 


def fit_neighbors(embN,n):
    X = embN(n,maxN=9)
    CC = list(Compositions(n))
    from sklearn.neighbors import NearestNeighbors as NN
    neigh = NN()
    neigh = neigh.fit(X)
    return neigh,CC


def get_neighbors(emb,neigh,CC,C,n_neighbors=5,reverse=True,verbose=False):
    v = emb(C,axis=0).transpose()
    dists, inds = (neigh.kneighbors(v, n_neighbors, return_distance=True))
    if verbose:
        print("dists = ",dists)
        print("inds = ", inds)
    ll = []
    for i in inds[0]:
        ll.append(CC[i])
    if reverse:
        return ll[-1::-1]
    else:    
        return ll    

neigh,CC = fit_neighbors(phiN,8)
X = phiN(4,maxN=8)
print(matrix(X).rank())
ll = get_neighbors(phi,neigh,CC,[2,1,1,2,1,1],n_neighbors=15,verbose=True,reverse=True)
print(ll)


class SinePlayer:
    def __init__(self,fn):
        from scipy.io.wavfile import write
        self.write = write
        self.wav = open(fn,"wb")
        self.p = pyaudio.PyAudio()
        self.fs = 44100
        self.wav_sample = []
        self.stream = self.p.open(format=pyaudio.paFloat32,channels=1,rate=self.fs,output=True)
        
    def play(self,frequency,duration,volume):
        f = frequency
        samples = (volume*np.sin(2*np.pi*np.arange(self.fs*duration)*f/self.fs)).astype(np.float32).tobytes()    
        self.stream.write(samples)

    def playMultiple(self,frequencies,duration,volumes):
        samples = []
        for i in range(len(frequencies)):
            f = frequencies[i]
            v = volumes[i]
            sample = (v*np.sin(2*np.pi*np.arange(self.fs*duration)*f/self.fs))
            samples.append(sample)
        final_sample = np.sum(samples,axis=0)/len(frequencies)    
        final_sample_tb = final_sample.astype(np.float32).tobytes()    
        self.stream.write(final_sample_tb)
        self.wav_sample.extend(final_sample.tolist())
        
        
    def stop(self):
        self.stream.stop_stream()
        self.stream.close()
        self.p.terminate()
        print("writing to wav...")
        self.write(self.wav,self.fs,np.array(self.wav_sample))
        self.wav.close()

def getRational(alpha,k):
    x = RDF(alpha**k).n(53)
    #print(x.nearby_rational(max_error=0.001*x))
    return x.nearby_rational(max_error=0.01*x)

def kernPitch(p1,p2):
    x = p1-p2
    q = getRational(2**(1.0/18.0),x)
    a,b = q.numerator(),q.denominator()
    return gcd(a,b)**2/(a*b)


# parametric knots
def lissajous(t,nx,ny,nz,phix,phiy):
    if not gcd(nx,ny)==gcd(nx,nz)==gcd(ny,nz)==1:
        return None
    x = np.cos(nx*t+phix)
    y = np.cos(ny*t+phiy)
    z = np.cos(nz*t)
    return x,y,z
    
def trefoil(t):
    import numpy as np
    x = np.sin(t)+2*np.sin(2*t)
    y = np.cos(t)-2*np.cos(t)
    z = -np.sin(3*t)
    return x,y,z

def kernChord(c1,c2):
    k = 1/(len(c1)*len(c2))*sum([kernPitch(x,y) for x in c1 for y in c2])*1.0
    #print(k)
    return k


def getNearestChords(t1,chords,n_neighbors=1,reverse=True):
    s = [x[1] for x in sorted([ (kernChord(t1,t2),t2) for t2 in chords],reverse=True)]
    #print(s)
    if reverse:
        return s[:n_neighbors]
    else:
        return s[(n_neighbors-1)::-1]

import numpy as np

pitchToFreq = dict([])
N = 18
alpha = 2**(1.0/N)
freqToPitch = dict([])
for i in range(N*7):
    #print(gcd(a,N)*1.0/N)
    print(i,alpha**i,55*alpha**i)
    #sw = synthesizer.generate_constant_wave(freq, 0.25+1/getRational(a))
    #sw = synthesizer.generate_constant_wave(freq, 0.25+1/(a))
    freq = 55*alpha**i
    #player.play_wave(synthesizerTriangle.generate_chord(chord,0.125))
    if freq>=220 and freq<=880:
        freqToPitch[freq] = i
        pitchToFreq[i] = freq
    
    #player.play_wave(synthesizer.generate_chord(chord,0.125*a))
    #print(t,chord,0.125*a)        
        
sp = SinePlayer("./knots/sines_4_1_2.wav")
c = 0
pitch = list(pitchToFreq.keys())[0]
seq = [sum(Integer(d).digits(2))+1 for d in range(1,256+1)]
lastFreq =440.0
xdurs = [4,1,1,2]
for i in range(len(seq)):
    xdurs = list(get_neighbors(phi,neigh,CC,xdurs,n_neighbors=seq[i],reverse=seq[i]%2)[-1])
    print(xdurs)
    for duration in xdurs:
        duration = duration/sum(xdurs)
        volume = duration
        nch = getNearestChords([pitch],[[p] for p in pitchToFreq.keys()],n_neighbors=seq[i],reverse=seq[i]%2)
        freqs = [pitchToFreq[pp[0]] for pp in nch]
        print(seq[i],seq[i]%2, freqs,duration,volume)
        sp.playMultiple(freqs,duration,len(freqs)*[volume])
        pitch = nch[-1][0]    
sp.stop()

8
n =  8
A = 23205
P_n =  30030
dists =  [[0.         1.         1.15470054 1.26491106 1.29099445 1.30930734
  1.34164079 1.34839972 1.35873244 1.36277029 1.3662601  1.37198868
  1.38013112 1.38169856 1.38675049]]
inds =  [[ 68   4 100  84  36  76  20  64  70  12 116  69 108   0   6]]
[[1, 1, 1, 1, 3, 1], [1, 1, 1, 1, 1, 1, 1, 1], [3, 3, 1, 1], [2, 1, 1, 2, 2], [4, 2, 1, 1], [1, 1, 1, 3, 1, 1], [2, 1, 1, 3, 1], [2, 1, 1, 1, 1, 1, 1], [1, 1, 2, 2, 1, 1], [2, 1, 3, 1, 1], [1, 2, 1, 2, 1, 1], [2, 2, 2, 1, 1], [3, 1, 2, 1, 1], [1, 1, 1, 1, 2, 1, 1], [2, 1, 1, 2, 1, 1]]
0 1.00000000000000 55.0000000000000
1 1.03925922603184 57.1592574317514
2 1.08005973889231 59.4032856390768
3 1.12246204830937 61.7354126570155
4 1.16652903957612 64.1590971766864
5 1.21232606681354 66.6779336747449
6 1.25992104989487 69.2956577442180
7 1.30938457517497 72.0161516346235
8 1.36079000017438 74.8434500095907
9 1.41421356237309 77.7817459305202
10 1.46973449227560 80.8353970751579
11 1.52743513091464 84.00893220

3 1 [576.129213076988, 288.064606538494, 228.637029727006] 1/8 1/8
3 1 [228.637029727006, 457.274059454011, 576.129213076988] 1/4 1/4
n =  8
A = 5005
P_n =  30030
[3, 1, 1, 1, 2]
3 1 [576.129213076988, 288.064606538494, 228.637029727006] 3/8 3/8
3 1 [228.637029727006, 457.274059454011, 576.129213076988] 1/8 1/8
3 1 [576.129213076988, 288.064606538494, 228.637029727006] 1/8 1/8
3 1 [228.637029727006, 457.274059454011, 576.129213076988] 1/8 1/8
3 1 [576.129213076988, 288.064606538494, 228.637029727006] 1/4 1/4
n =  8
A = 5005
P_n =  30030
[4, 1, 1, 2]
4 0 [288.064606538494, 576.129213076988, 457.274059454011, 228.637029727006] 1/2 1/2
4 0 [288.064606538494, 576.129213076988, 457.274059454011, 228.637029727006] 1/8 1/8
4 0 [288.064606538494, 576.129213076988, 457.274059454011, 228.637029727006] 1/8 1/8
4 0 [288.064606538494, 576.129213076988, 457.274059454011, 228.637029727006] 1/4 1/4
n =  8
A = 1001
P_n =  30030
[4, 1, 1, 2]
3 1 [228.637029727006, 457.274059454011, 576.129213076988] 1/2

5 1 [228.637029727006, 457.274059454011, 576.129213076988, 288.064606538494, 533.423469397959] 1/8 1/8
5 1 [533.423469397959, 266.711734698980, 672.071457602443, 423.378488233424, 228.637029727006] 1/4 1/4
n =  8
A = 10010
P_n =  30030
[1, 3, 1, 1, 2]
4 0 [288.064606538494, 576.129213076988, 457.274059454011, 228.637029727006] 1/8 1/8
4 0 [288.064606538494, 576.129213076988, 457.274059454011, 228.637029727006] 3/8 3/8
4 0 [288.064606538494, 576.129213076988, 457.274059454011, 228.637029727006] 1/8 1/8
4 0 [288.064606538494, 576.129213076988, 457.274059454011, 228.637029727006] 1/8 1/8
4 0 [288.064606538494, 576.129213076988, 457.274059454011, 228.637029727006] 1/4 1/4
n =  8
A = 2002
P_n =  30030
[1, 3, 1, 1, 2]
5 1 [228.637029727006, 457.274059454011, 576.129213076988, 288.064606538494, 533.423469397959] 1/8 1/8
5 1 [533.423469397959, 266.711734698980, 672.071457602443, 423.378488233424, 228.637029727006] 3/8 3/8
5 1 [228.637029727006, 457.274059454011, 576.129213076988, 288.064606538

4 0 [288.064606538494, 576.129213076988, 457.274059454011, 228.637029727006] 1/8 1/8
4 0 [288.064606538494, 576.129213076988, 457.274059454011, 228.637029727006] 1/4 1/4
n =  8
A = 5005
P_n =  30030
[4, 1, 1, 2]
4 0 [288.064606538494, 576.129213076988, 457.274059454011, 228.637029727006] 1/2 1/2


ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred


...

7 1 [533.423469397959, 266.711734698980, 672.071457602443, 423.378488233424, 228.637029727006, 237.613142556307, 846.756976466848] 1/4 1/4
n =  8
A = 455
P_n =  30030
[4, 2, 2]
4 0 [672.071457602443, 336.035728801221, 423.378488233424, 846.756976466848] 1/2 1/2
4 0 [672.071457602443, 336.035728801221, 423.378488233424, 846.756976466848] 1/4 1/4
4 0 [672.071457602443, 336.035728801221, 423.378488233424, 846.756976466848] 1/4 1/4
n =  8
A = 91
P_n =  30030
[4, 2, 2]
5 1 [846.756976466848, 423.378488233424, 336.035728801221, 672.071457602443, 362.938661507533] 1/2 1/2
5 1 [362.938661507533, 725.877323015066, 457.274059454011, 288.064606538494, 846.756976466848] 1/4 1/4
5 1 [846.756976466848, 423.378488233424, 336.035728801221, 672.071457602443, 362.938661507533] 1/4 1/4
n =  8
A = 91
P_n =  30030
[4, 2, 2]
5 1 [362.938661507533, 725.877323015066, 457.274059454011, 288.064606538494, 846.756976466848] 1/2 1/2
5 1 [846.756976466848, 423.378488233424, 336.035728801221, 672.071457602443, 362.9

5 1 [783.990871963498, 391.995435981749, 311.126983722081, 622.253967444162, 336.035728801221] 1/8 1/8
5 1 [336.035728801221, 672.071457602443, 846.756976466848, 423.378488233424, 266.711734698980] 1/4 1/4
n =  8
A = 5005
P_n =  30030
[3, 1, 1, 1, 2]
5 1 [266.711734698980, 533.423469397959, 672.071457602443, 336.035728801221, 622.253967444162] 3/8 3/8
5 1 [622.253967444162, 311.126983722081, 246.941650628062, 783.990871963498, 493.883301256124] 1/8 1/8
5 1 [493.883301256124, 246.941650628062, 622.253967444162, 391.995435981749, 783.990871963498] 1/8 1/8
5 1 [783.990871963498, 391.995435981749, 311.126983722081, 622.253967444162, 336.035728801221] 1/8 1/8
5 1 [336.035728801221, 672.071457602443, 846.756976466848, 423.378488233424, 266.711734698980] 1/4 1/4
n =  8
A = 5005
P_n =  30030
[3, 2, 1, 2]
6 0 [598.747600076726, 622.253967444162, 336.035728801221, 672.071457602443, 533.423469397959, 266.711734698980] 3/8 3/8
6 0 [598.747600076726, 622.253967444162, 336.035728801221, 672.07145760

6 0 [754.374704910704, 256.636388706746, 407.384873406408, 814.769746812815, 646.683176601263, 323.341588300632] 1/8 1/8
6 0 [754.374704910704, 256.636388706746, 407.384873406408, 814.769746812815, 646.683176601263, 323.341588300632] 1/4 1/4
n =  8
A = 715
P_n =  30030
[3, 2, 1, 2]
7 1 [323.341588300632, 646.683176601263, 814.769746812815, 407.384873406408, 256.636388706746, 754.374704910704, 725.877323015066] 3/8 3/8
7 1 [725.877323015066, 362.938661507533, 288.064606538494, 576.129213076988, 311.126983722081, 323.341588300632, 457.274059454011] 1/4 1/4
7 1 [457.274059454011, 228.637029727006, 576.129213076988, 362.938661507533, 725.877323015066, 288.064606538494, 533.423469397959] 1/8 1/8
7 1 [533.423469397959, 266.711734698980, 672.071457602443, 423.378488233424, 228.637029727006, 237.613142556307, 846.756976466848] 1/4 1/4
n =  8
A = 715
P_n =  30030
[3, 2, 1, 2]
5 1 [846.756976466848, 423.378488233424, 336.035728801221, 672.071457602443, 362.938661507533] 3/8 3/8
5 1 [362.93866150

7 1 [598.747600076726, 299.373800038363, 237.613142556307, 754.374704910704, 475.226285112615, 256.636388706746, 266.711734698980] 1/4 1/4
n =  8
A = 13
P_n =  30030
[6, 2]
5 1 [266.711734698980, 533.423469397959, 672.071457602443, 336.035728801221, 622.253967444162] 3/4 3/4
5 1 [622.253967444162, 311.126983722081, 246.941650628062, 783.990871963498, 493.883301256124] 1/4 1/4
n =  8
A = 13
P_n =  30030
[4, 2, 2]
6 0 [311.126983722081, 783.990871963498, 391.995435981749, 622.253967444162, 246.941650628062, 493.883301256124] 1/2 1/2
6 0 [311.126983722081, 783.990871963498, 391.995435981749, 622.253967444162, 246.941650628062, 493.883301256124] 1/4 1/4
6 0 [311.126983722081, 783.990871963498, 391.995435981749, 622.253967444162, 246.941650628062, 493.883301256124] 1/4 1/4
n =  8
A = 91
P_n =  30030
[6, 2]
6 0 [311.126983722081, 783.990871963498, 391.995435981749, 622.253967444162, 246.941650628062, 493.883301256124] 3/4 3/4
6 0 [311.126983722081, 783.990871963498, 391.995435981749, 622.253

n =  8
A = 91
P_n =  30030
[4, 1, 1, 2]
8 0 [493.883301256124, 698.456462866008, 725.877323015066, 246.941650628062, 391.995435981749, 783.990871963498, 622.253967444162, 311.126983722081] 1/2 1/2
8 0 [493.883301256124, 698.456462866008, 725.877323015066, 246.941650628062, 391.995435981749, 783.990871963498, 622.253967444162, 311.126983722081] 1/8 1/8
8 0 [493.883301256124, 698.456462866008, 725.877323015066, 246.941650628062, 391.995435981749, 783.990871963498, 622.253967444162, 311.126983722081] 1/8 1/8
8 0 [493.883301256124, 698.456462866008, 725.877323015066, 246.941650628062, 391.995435981749, 783.990871963498, 622.253967444162, 311.126983722081] 1/4 1/4
n =  8
A = 1001
P_n =  30030
[4, 1, 1, 2]
7 1 [311.126983722081, 622.253967444162, 783.990871963498, 391.995435981749, 246.941650628062, 725.877323015066, 698.456462866008] 1/2 1/2
7 1 [698.456462866008, 349.228231433004, 277.182630976872, 880.000000000000, 554.365261953744, 299.373800038363, 311.126983722081] 1/8 1/8
7 1 [311.126

In [158]:
def HH1(x,i):
    q = getRational(x,i)
    a,b = q.numerator(),q.denominator()
    #print(x,q,a,b)
    #return gcd(a,b)**2/(a*b)
    return gcd(a,b)/(a+b)

def HH2(x,i):
    q = getRational(x,i)
    a,b = q.numerator(),q.denominator()
    #print(x,q,a,b)
    return gcd(a,b)**2/(a*b)
    #return gcd(a,b)/(a+b)



alpha = 2**(1.0/12.0)

def sH(alpha,n):
    return sum([HH1(alpha,i) for i in range(n)]).n(),sum([HH2(alpha,i) for i in range(n)]).n()

def lH(alpha,n):
    return sorted([(HH1(alpha,i),i) for i in range(n)],reverse=True),sorted([(HH2(alpha,i),i) for i in range(n)],reverse=True)

for n in [12,18]:
    print(n,lH(2**(1.0/n),n))

for n in range(1,101):
    for k in range(2,20):
        x,y  = sH(k**(1.0/n),n)
        a,b = sH(k**(1.0/12.0),12)
        if x <= a and y <= b and n>=12:
            print(k,n,x,y)

12 ([(1/2, 0), (1/5, 7), (1/7, 5), (1/8, 9), (1/9, 4), (1/11, 3), (1/13, 8), (1/17, 2), (1/23, 11), (1/25, 10), (1/29, 6), (1/31, 1)], [(1, 0), (1/6, 7), (1/12, 5), (1/15, 9), (1/20, 4), (1/30, 3), (1/40, 8), (1/72, 2), (1/120, 11), (1/144, 10), (1/204, 6), (1/240, 1)])
18 ([(1/2, 0), (1/9, 6), (1/13, 12), (1/13, 4), (1/17, 16), (1/17, 3), (1/19, 14), (1/20, 5), (1/23, 7), (1/25, 15), (1/25, 2), (1/26, 8), (1/29, 13), (1/29, 9), (1/32, 17), (1/32, 10), (1/33, 11), (1/43, 1)], [(1, 0), (1/20, 6), (1/40, 12), (1/42, 4), (1/66, 16), (1/72, 3), (1/84, 14), (1/99, 5), (1/130, 7), (1/144, 15), (1/156, 2), (1/165, 8), (1/198, 13), (1/204, 9), (1/231, 17), (1/247, 10), (1/260, 11), (1/462, 1)])
2 12 1.45584303521857 1.46323529411765
3 12 1.14329556353317 1.22196024220018
4 12 1.49680133766881 1.75653127917834
5 12 1.20770694838213 1.31480024449332
6 12 1.03651875932602 1.14449863335063
7 12 1.20475919271291 1.34952232867565
8 12 1.59990657180665 1.96413828689370
9 12 1.33972377444561 1.5782767